In [1]:
import requests

解析一级网站，获取不同国家的列表

In [2]:
url = "https://www.directoriodefabricas.com/productos/fabricantes-de-envases.html"
response = requests.get(url).content

In [3]:
from bs4 import BeautifulSoup
soup= BeautifulSoup(response, 'lxml')

In [4]:
ListTitle = soup.find_all("a",attrs={"class":"dir-IndicLiLinks1"})

In [5]:
import re
pattern = re.compile(r">(.*?)</a>")
List = pattern.findall(str(ListTitle))

In [6]:
pattern_link=re.compile(r"href=\"(.*?)\"")
List_link = pattern_link.findall(str(ListTitle))

把国家与对应的网址，整合进列表

In [7]:
Dict1 ={}
for i in range(len(List)):
    Dict1[List[i]]=List_link[i]

定义爬取各详细页资料的方法

In [8]:
def get_data(url):
    list2=[]
    response = requests.get(url).content
    soup = BeautifulSoup(response,'lxml')
    Data_Nombre = soup.find_all("div",attrs={"class":"dir-LinkDiv1"})
    pattern_Nombre = re.compile(r">(.*?)</div>")
    Nombre = pattern_Nombre.findall(str(Data_Nombre))
    Data_Sitio1 = soup.find_all("a",attrs={"class":"dir-LinksDivDecor1"})
    Data_Sitio2 = soup.find_all("a",attrs={"class":"dir-WebLinkDivDecor1"})
    # 据观察，网址的class类型 有两种变化，需要进行判断，选择对应的class进行解析
    pattern_Sitio=re.compile(r"href=\"(.*?)\"")
    if Data_Sitio1:
        Link = pattern_Sitio.findall(str(Data_Sitio1))
    else:
        Link = pattern_Sitio.findall(str(Data_Sitio2))
    Data_Des = soup.find_all("div",attrs={"class":"dir-LinkDiv2"})
    pattern_Des=re.compile(r">(.*?)</div>")
    Des = pattern_Des.findall(str(Data_Des))
    try:
        for i in range(len(Link)):
            list2.append({"Nombre":Nombre[i],"Link":Link[i],"Des":Des[i]})
    except:
        print('something wrong')
    return list2

In [9]:
import csv

In [10]:
for key, value in Dict1.items():
    url = value
    list2 = get_data(url)
    fieldnames = ['Nombre', 'Link','Des']
    with open("./"+key+'.csv','w',encoding='utf-8') as file:
        writer = csv.DictWriter(file, fieldnames=fieldnames)
        for item in list2:
            writer.writerow(item)
        file.close()